In [1]:
import numpy as np
import pandas as pd

from pandas import Series, DataFrame

In [2]:
total_data = pd.read_csv('full_log.log', sep=" ", header=None)

In [3]:
total_data['time'] = total_data[3] + ' ' + total_data[4]
total_data['method'], total_data['url'], total_data['protocol'] = total_data[5].str.split(' ', 2).str
total_data = total_data.drop([1, 2, 3, 4, 5, 7], axis=1)

In [4]:
df_aux = total_data[total_data.isna().any(axis=1)]

In [5]:
df = total_data.dropna()

print(len(total_data), '-', len(df_aux), '=', len(df))
print('then everything is ok') if (len(total_data) - len(df_aux) == len(df)) else print('then everything is wrong')

7823 - 60 = 7763
then everything is ok


In [6]:
df_aux = df.copy()

df_aux[[8]] = df[[8]].replace('-', pd.np.nan)
df_aux = df_aux[~df_aux.isna().any(axis=1)]

print('there was', len(df_aux), 'http_referer notnull records')

df = df.drop([8], axis=1)

there was 110 http_referer notnull records


In [7]:
df_aux = df.copy()
df_aux[[10]] = df[[10]].replace('-', pd.np.nan)

df_aux = df_aux[~df_aux.isna().any(axis=1)]

print('there was', len(df_aux), 'http_x_forwarded_for notnull records')

df = df.drop([10], axis=1)

there was 13 http_x_forwarded_for notnull records


In [8]:
df_aux = df.copy()

df[['method']] = df[['method']].replace('CONNECT', pd.np.nan)
df[['method']] = df[['method']].replace('HEAD', pd.np.nan)
df[['method']] = df[['method']].replace('OPTIONS', pd.np.nan)
df[['method']] = df[['method']].replace('PROPFIND', pd.np.nan)
df[['method']] = df[['method']].replace('REQMOD', pd.np.nan)

df[['url']] = df[['url']].replace('', pd.np.nan)

df = df.dropna()

print('there was', len(df_aux) - len(df), 'CONNECT, HEAD, OPTIONS, PROPFIND, REQMOD or EMPTY URL records')

there was 44 CONNECT, HEAD, OPTIONS, PROPFIND, REQMOD or EMPTY URL records


In [9]:
df.columns = ['remote_addr', 'status', 'http_user_agent', 'time_local', 'request_method', 'request_url', 'request_protocol']
df_aux.columns = ['remote_addr', 'status', 'http_user_agent', 'time_local', 'request_method', 'request_url', 'request_protocol']

df = df.drop(['remote_addr', 'time_local', 'request_protocol'], axis=1)

In [10]:
df[['http_user_agent']] = df[['http_user_agent']].replace('-', '<NO_DATA>')
df = df[['http_user_agent', 'request_method', 'status', 'request_url']]

In [11]:
# Deleting 3 bad request manually
df = df.drop([887, 3671, 3759], axis=0)
df = df.reset_index(drop=True)
df['is_evil'] = ''

In [12]:
cont = 0
cont2 = 0
cont3 = 0
cont4 = 0
cont_t = 0
q_str = 'php'
q_str2 = 'busybox'
q_str3 = 'xml'
q_str4 = 'robots.txt'
for i in df.itertuples():
    is_counted = False
    if q_str in i[4]:
        cont += 1
        df.at[i[0], 'is_evil'] = '1'
        if not is_counted:
            cont_t += 1
            is_counted = True
    if q_str2 in i[4]:
        cont2 += 1
        df.at[i[0], 'is_evil'] = '1'
        if not is_counted:
            cont_t += 1
            is_counted = True
    if q_str3 in i[4]:
        cont3 += 1
        df.at[i[0], 'is_evil'] = '1'
        if not is_counted:
            cont_t += 1
            is_counted = True
    if q_str4 in i[4]:
        cont4 += 1
        df.at[i[0], 'is_evil'] = '1'
        if not is_counted:
            cont_t += 1
            is_counted = True

        
print('there are', cont, q_str, 'urls,', cont2, q_str2, 'urls,', cont3, q_str3, 'urls,', cont4, q_str4, 'urls')
print('total evilized records are', cont_t, 'from', len(df))
print('leaving', len(df) - cont_t, 'records to edit')

there are 6757 php urls, 4 busybox urls, 1457 xml urls, 51 robots.txt urls
total evilized records are 6866 from 7719
leaving 853 records to edit


In [13]:
df = df.sort_values('is_evil')

In [14]:
import openpyxl

df.to_excel('log_data.xlsx')

print('total records:',len(df))

total records: 7719
